# Project: Cel-shading

## Project Description
The purpose of this project was to create an algorithm that takes in images and transforms the people in the images to a cartoony style reminiscent of the art-style in the 2009 video game "Borderlands" and old-school cel-shading rendering.
The pipeline from an unprocessed image to the final result consists of a few key steps:
    - Use semantic segmentation to only affect people
    - Normalize brightness, or Value from HSV, into a set number of levels of brightness
    - Use edge-detection to find the edges of each area of brightness
    - Draw over these edges in the normalized image to outline shapes and contours in the image.

## The Images
Any image with a person should work, but the more clearly the person is standing out from the background, and the more differences in lighting on the person, the better.¨
Here are a few example images, the first of which will be passed through each step of the algorithm so you can see what is happening.

In [ ]:
# Imports
import torch
from torchvision import transforms
import cv2
from PIL import Image
import numpy as np
from segmentation import blur_object_pixels
from normalize_HSV import normalize_brightness, apply_mask, np2img
from draw_outlines import draw_outlines
from full_pipeline import full_pipeline

# Loading the first image, to be used onwards
filename = 'data/image6.jpg'
input_image = Image.open(filename)
input_image = input_image.convert("RGB")
display(input_image)

# Two more example images:
example1 = 'data/image7.jpg'
example1 = Image.open(example1)
display(example1)

example2 = 'data/image8.jpg'
example2 = Image.open(example2)
display(example2)

## Segmentation and Object Detection

The first step is to figure out where in the image a person is. This is done by using a semantic segmentation algorithm that utilizes the deeplabv3_resnet101 model from pytorch. The image is smoothed with a gaussian kernel to make the outline of the person smoother, and to improve the model's segmentation capabilities. Only the area with the person is blurred.

The model predicts the locations of a wide range of objects, such as people, airplanes, bicycles, cars, buses, horses and motorbikes. The prediction for the person-object is what we're interested in, and this is readily available through a produced python dictionary.

First, the 


In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=True)
input_image_blurred, person_mask = blur_object_pixels(model, input_image, ['person'], sigma=3, show_object_list=False, concat=False, scale=1)
display(person_mask)

## Normalize lighting via HSV-values
Now that we've extracted the outline of the person, it's time to normalize the brightness into a few distinct levels.
This is done by converting the image from the normal RGB-format over to a HSV-format, where the V - Value, represents the brightness of each pixel. By comparing the value of each pixel in the image with a range of thresholds, and setting the value to the last threshold it's brighter than, we get a variety of masks that show the different brightness areas of the image. By setting the "high value" of the masks to 255/N where N is the number of brightness thresholds, we can simply add the masks together to create the brightness mask of the image, as seen below.

Once we have the brightness mask, we can apply it by converting the blurred image from earlier into HSV, and setting the Value of each pixel to the corresponding Value in the mask, within the confines of the segmented person.

In [ ]:
ms = normalize_brightness(image=input_image_blurred, N=4, object_mask=person_mask)  # normalize brightness into N different levels
img = ms["all"]  # get the mask with all the brightness levels composited
display(img)  # display the full brightness mask
masked = apply_mask(im=input_image, mask=img)  # apply the brightness mask to the image
display(masked)  # display the normalized image

## Edge Detection
Now that we have a person with normalized brightness, we can apply an edge detection algorithm to find the outlines of the areas of similar brightness, and then draw over them to complete the cartoonish look.
The openCV library has multiple functions for edge detection, and through testing, the best one seems to be the canny edge detection.

Canny edges is a multi-step algorithm that can be read about [here](https://en.wikipedia.org/wiki/Canny_edge_detector), but the most important things is knowing the input and outputs.
The inputs are the image itself, and two thresholds used by the algorithm to determine edges.

The first image shows the edges detected in the image above, and the second image is when filtered by the segmentation mask.

In [ ]:
img = np.array(masked)[:, :, ::-1].copy()  # Placeholder for the detected edges

# Canny Edge Detection
edges = Image.fromarray(cv2.Canny(image=img, threshold1=0, threshold2=200)) # Canny Edge Detection
display(edges)

# Filter detected edges by the segmented person mask
outlines = Image.composite(edges, person_mask, person_mask)
display(outlines)

## Drawing over detected edges
Finally, all that remains is to use the detected edges to draw lines onto the normalized, segmented image. This is done simply enough by iterating over the pixels, and setting the RGB-values to 0 where the edges are detected.
The lines can be made thicker by drawing the line in an area round the edges, but empirically the nicest looking result is when the line is only 1px wide.

In [ ]:
line_width = 1  # width of line to draw over detected edges
final_image = np2img(draw_outlines(masked,outlines, line_width))  # draw lines in <masked> following detected edges in <outlines>
display(final_image)  # display the final image

## Try it yourself!
Below is all the relevant parameters and a function call that runs the entire pipeline so you can explore different results and outputs!

In [ ]:
image_path = 'data/image6.jpg'  # path to your image
segmentation_model ='deeplabv3_resnet101' 
blur_sigma = 3 
processing_scale = 1 
brightness_segments = 4 
canny_lower_threshold = 0 
canny_upper_threshold = 200 
line_width = 1
full_pipeline(image_path=image_path, segmentation_model=segmentation_model, blur_sigma=blur_sigma, processing_scale=processing_scale, brightness_segments=brightness_segments, canny_lower_threshold=canny_lower_threshold, canny_upper_threshold=canny_upper_threshold, line_width=line_width)